In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import torch

# Linear Regression

In linear regression we have a 
- continuous one-dimensional target $y$ 
- continuous D-dimensional input $x$ 

related by a linear mapping

$$
b + \sum_{d=1}^D w_d x_d = f_\theta(x)  \rightarrow y
$$

> The model is specified by $\theta=(b, w_1, w_2, \ldots, w_D)$

Typically, we fit this model by 
$$
\min_\theta\sum_n \left(y_n - f_\theta(x_n) \right)^2 = (Y - \Phi \theta)^T (Y - \Phi \theta)
$$

whose solution is

$$
\theta = (\Phi^T \Phi)^{-1} \Phi^T Y,
$$

where $\Phi  = \begin{pmatrix} 1 & x_{11} & x_{12} & \ldots & x_{1D} \\ 
1 & x_{21} & x_{22} & \ldots & x_{2D} \\
1 & \vdots & \vdots & \ddots & \vdots \\
1 & x_{N1} & x_{N2} & \ldots & x_{ND} \end{pmatrix}$,  $Y = \begin{pmatrix} y_1 \\ y_2 \\ \vdots \\ y_N \end{pmatrix}$ and  $\theta =  \begin{pmatrix} b \\ w_1 \\ \vdots \\ w_D \end{pmatrix}$

> This is known as the ordinary least squares (OLS) solution

#### Note: Linear regression is *linear on the parameters*

If we apply transformations we obtain the same solution. The only difference is in $\Phi$

For example
- Polynomial basis regression $f_\theta(x) = \sum_d w_d x^d + b$ 
- Sine-wave basis regression $f_\theta(x) = \sum_d \alpha_d \cos(2\pi d f_0 x)  + \sum_d \beta_d \sin(2\pi d f_0 x) + c$ 

### Probabilistic linear regression

We can assume that observations are noisy and write

$$
\begin{align}
y &= f_\theta(x) + \epsilon \nonumber \\
&= b + \sum_{d=1}^D w_d x_d   + \epsilon, \nonumber
\end{align}
$$

If the noise is independent and Gaussian distributed (iid) with variance $\sigma_\epsilon^2$ then

$$
p(y|x, \theta) = \mathcal{N}\left(y| f_\theta(x) , \sigma_\epsilon^2 \right)
$$

Additionally, we may want to discourage large values of $\theta~$ by placing a prior

$$
p(\theta) = \mathcal{N}(0, \Sigma_\theta)
$$

The prior on the parameters gives us the space of possible models (before presenting data)

In [ ]:
line_x = np.linspace(-5, 5, num=100)[:, None].astype('float32') #100x1

sw, sb = 5., 5.
fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)
for i in range(100):
    linear_layer = torch.nn.Linear(1, 1)
    torch.nn.init.normal_(linear_layer.weight, 0.0, sw)
    torch.nn.init.normal_(linear_layer.bias, 0.0, sb)
    #y = W*x + b
    line_y = linear_layer(torch.from_numpy(line_x)).detach().numpy()
    ax.plot(line_x, line_y, c='tab:blue', alpha=0.25)

We constraint the space of solutions by presenting data

### Point-estimate solution (MAP)

For a dataset $\mathcal{D} = \{ (x_1, y_1), (x_2, y_2), \ldots, (x_N, y_N) \}$

The Maximum a posteriori estimator of $\theta~$ is given by
$$
\begin{align}
\hat \theta &= \text{arg}\max_\theta \log p(\mathcal{D}| \theta, \sigma_\epsilon^2) ~ \mathcal{N} (\theta|0, \Sigma_\theta) \nonumber  \\
&= \text{arg}\min_\theta  \frac{1}{2\sigma_\epsilon^2} (Y-\Phi\theta)^T (Y - \Phi\theta) + \frac{1}{2} \theta^T \Sigma_\theta^{-1} \theta  \nonumber
\end{align}
$$
where the log likelihood is
$$
\log p(\mathcal{D}| \theta, \sigma_\epsilon^2) = \sum_{n=1}^N \log \mathcal{N}(y_n|f_\theta(x_n),\sigma_\epsilon^2)
$$
and the result is
$$
\hat \theta = (\Phi^T \Phi + \lambda )^{-1} \Phi^T Y
$$
where $\lambda = \sigma_\epsilon^2 \Sigma_\theta^{-1}$

> This is the ridge regression or **regularized least squares** solution

What happens if the variance of the prior tends to infinite (uninformative prior)?


### Bayesian solution for the parameters

In this case we want the posterior of $\theta~$ given the dataset

Assuming that we know $\sigma_\epsilon$

$$
p(\theta|\mathcal{D}, \sigma_\epsilon^2) \propto  \mathcal{N}(Y| \Phi \theta, I\sigma_\epsilon^2) \mathcal{N}(\theta| \theta_0, \Sigma_{\theta_0})
$$

The likelihood is normal and the prior is normal, so
$$
p(\theta|\mathcal{D}, \sigma_\epsilon^2) \propto \frac{1}{Z} \exp \left ( -\frac{1}{2\sigma_\epsilon^2} (Y-\Phi\theta)^T (Y - \Phi\theta)  - \frac{1}{2} (\theta - \theta_{0})^{T} \Sigma_{\theta_0}^{-1} (\theta - \theta_0)\right)
$$

and (with a bit of algebra) it can be shown that this corresponds to a normal distribution 

$$
p(\theta|\mathcal{D}, \sigma_\epsilon^2) = \mathcal{N}(\theta|\theta_1, \Sigma_{\theta_1} )
$$

with parameters 
$$
\Sigma_{\theta_1} = \sigma_\epsilon^2 (\Phi^T \Phi + \sigma_\epsilon^2  \Sigma_{\theta_0}^{-1})^{-1}
$$
$$
\theta_1 = \Sigma_{\theta_1} \Sigma_{\theta_0}^{-1} \theta_{0} + \frac{1}{\sigma_\epsilon^2} \Sigma_{\theta_1} \Phi^T y
$$

> **Iterative framework:** We can present data and update the distribution of $\theta~$

#### Fitting a line

We assume a zero-mean and diagonal covariance normal prior

In [ ]:
# Initialization
mw, mb = 0., 0.
sw, sb = 5., 5.
So = np.diag(np.array([sb, sw])**2)
mo = np.array([mb, mw])
seps = 1. # What happens if this is larger/smaller?

The empirical distribution of $\theta$

In [ ]:
theta_plot = np.random.multivariate_normal(mo, So, size=10000)

import corner
figure = corner.corner(theta_plot, smooth=1.,
                       labels=["b", "w"], bins=20, 
                       quantiles=[0.16, 0.5, 0.84], range=[(-8, 8), (-8, 8)],
                       show_titles=True, title_kwargs={"fontsize": 12})

We observe data at $x=2$, $y=0$ and we update the parameters

In [ ]:
#Update
Phi = np.array([[1.0, 2.0]])
y = np.array([0.0])
Sn = seps**2*np.linalg.inv(np.dot(Phi.T, Phi) +  seps**2*np.linalg.inv(So))
mn = np.dot(Sn, np.linalg.solve(So, mo)) + np.dot(Sn, np.dot(Phi.T, y))/seps**2
display(Sn, mn)

with this the space of possible models is constrained

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)

for i in range(100):
    linear_layer = torch.nn.Linear(1, 1)
    rparam = torch.from_numpy(np.random.multivariate_normal(mn, Sn).astype('float32'))
    linear_layer.weight.data = rparam[1].reshape(-1, 1)
    linear_layer.bias.data = rparam[0].reshape(-1, 1)
    line_y = linear_layer(torch.from_numpy(line_x)).detach().numpy()
    ax.plot(line_x, line_y, c='tab:blue', alpha=0.25)

ax.errorbar(2, 0, xerr=0, yerr=2*seps, fmt='none', c='k', zorder=100);

and the updated empirical distribution is 

In [ ]:
theta_plot = np.random.multivariate_normal(mn, Sn, size=10000)

figure = corner.corner(theta_plot, smooth=1.,
                       labels=["b", "w"], bins=20, 
                       quantiles=[0.16, 0.5, 0.84], range=[(-8, 8), (-8, 8)],
                       show_titles=True, title_kwargs={"fontsize": 12})

Let's assume that we observe a additional data point at $x=-2$, $y=-2$

In [ ]:
# Initialization
So = Sn
mo = mn
#Update
Phi = np.array([[1.0, -2.0]])
y = np.array([-2.0])
Sn = seps**2*np.linalg.inv(np.dot(Phi.T, Phi) +  seps**2*np.linalg.inv(So))
mn = np.dot(Sn, np.linalg.solve(So, mo)) + np.dot(Sn, np.dot(Phi.T, y))/seps**2
display(Sn, mn)

The space of possible models is further reduced

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)

for i in range(100):
    linear_layer = torch.nn.Linear(1, 1)
    rparam = torch.from_numpy(np.random.multivariate_normal(mn, Sn).astype('float32'))
    linear_layer.weight.data = rparam[1].reshape(-1, 1)
    linear_layer.bias.data = rparam[0].reshape(-1, 1)
    line_y = linear_layer(torch.from_numpy(line_x)).detach().numpy()
    ax.plot(line_x, line_y, c='tab:blue', alpha=0.2)

ax.errorbar(2, 0, xerr=0, yerr=2*seps, fmt='none', c='k', zorder=100);
ax.errorbar(-2, -2., xerr=0, yerr=2*seps, fmt='none', c='k', zorder=100);

And the empirical distribution is constrained even more

In [ ]:
theta_plot = np.random.multivariate_normal(mn, Sn, size=10000)

figure = corner.corner(theta_plot, smooth=1.,
                       labels=["b", "w"], bins=20, 
                       quantiles=[0.16, 0.5, 0.84], range=[(-8, 8), (-8, 8)],
                       show_titles=True, title_kwargs={"fontsize": 12})

### Bayesian solution for the predictions

Don't forget our goal

> We train the model to predict $y$ for new values of $x$

In the Bayesian setting we are interested in the **posterior predictive distribution**

This is obtained by marginalizing $\theta$

$$
\begin{align}
p(y | x, \mathcal{D}) &= \int p(y, \theta | x, \mathcal{D}) d\theta \nonumber \\
&= \int p(y| \theta, x, \mathcal{D}) p(\theta| \mathcal{D}) d\theta \nonumber \\
&= \int p(y| \theta, x) p(\theta| \mathcal{D}) d\theta, \nonumber 
\end{align}
$$
note that $y$ is conditionally independant on $\mathcal{D}$ given $\theta$

For our linear regression
$$
\begin{align}
p(y|x, \mathcal{D}, \sigma_\epsilon^2) &= \int p(y|f_\theta(x), \sigma_\epsilon^2) p(\theta| \theta_{N}, \Sigma_{\theta_N}) d\theta \nonumber \\
&= \mathcal{N}\left(y|f_{\theta_N} (x), \sigma_\epsilon^2 + x^T \Sigma_{\theta_N} x\right)
\end{align}
$$

**The posterior predictive is Gaussian** (convolution of gaussians is gaussian)

If we consider that $N$ samples were presented and that $\mu_0=0$ then 

$$
\theta_{N} =  (\Phi^T \Phi + \sigma_\epsilon^2 \Sigma_{\theta_0}^{-1})^{-1} \Phi^T y
$$

which is the MAP estimator, and

$$
\Sigma_{\theta_N} = \sigma_\epsilon^2 (\Phi^T \Phi + \sigma_\epsilon^2 \Sigma_{\theta_0}^{-1})^{-1}
$$


Finally, the variance (uncertainty) for the new $x$ is 
$$
\sigma^2(x) = \sigma_\epsilon^2 + x^T \Sigma_{\theta_N} x
$$

> The variance of the prediction has contribution from the noise (irreducible) and the model 

Uncertainty grows when we depart from the observed data points

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)

Phi_x = np.vstack(([1]*100, line_x[:, 0])).T
sx = np.sqrt(np.diag(seps**2 + np.dot(np.dot(Phi_x, Sn), Phi_x.T)))
ax.plot(line_x, np.dot(Phi_x, mn), '--')
ax.fill_between(line_x[:, 0], np.dot(Phi_x, mn)-2*sx, np.dot(Phi_x, mn)+2*sx, alpha=0.5)
ax.errorbar(2, 0, xerr=0, yerr=2*seps, fmt='none', c='k', zorder=100);
ax.errorbar(-2, -2., xerr=0, yerr=2*seps, fmt='none', c='k');

**Activity:**

See how the posterior predictive distribution changes with increasing/decreasing $\sigma_\epsilon$ and $\Sigma_{\theta_0}$

# Self-study

- [Chapter 18 of D. Barber's book](http://web4.cs.ucl.ac.uk/staff/D.Barber/pmwiki/pmwiki.php?n=Brml.Online)
- In all this we assumed $\sigma_\epsilon$ known. For a bayesian treatment with unknown noise variance we would use a normal inverse gamma prior

## Model Evidence for Bayesian Linear Regression

Next iteration

# PyTorch tutorial

Short pytorch tutorial in spanish:

- https://github.com/magister-informatica-uach/INFO267/blob/master/unidad1/3_redes_neuronales.ipynb

# Polynomial (linear) regression with Pytorch

Let's create synthetic data

We will fit this with a polynomial model

In [ ]:
# Synthetic data
se = 0.1
np.random.seed(0)
x = np.linspace(0, 1, num=20) #100x1
x_test = np.linspace(-0.05, 1.05, num=200)
f = lambda x : x*np.sin(10*x)

x = np.delete(x, slice(9, 14))
y = f(x) + se*np.random.randn(len(x))
fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)
ax.scatter(x, y);

x_torch = torch.from_numpy(x.astype('float32')).unsqueeze(1)
x_test = torch.from_numpy(x_test.astype('float32')).unsqueeze(1)
y_torch = torch.from_numpy(y.astype('float32')).unsqueeze(1)

The linear regressor model in PyTorch has one layer and no activation

We keep the degree of the pynomial as a free parameter

In [ ]:
class LinearRegressor(torch.nn.Module):
    def __init__(self, degree=10):
        super(LinearRegressor, self).__init__()
        assert degree>0, "Degree has to be greater than zero"
        assert type(degree)==int, "Degree has to be an integer"
        self.degree = degree
        self.linear = torch.nn.Linear(degree, 1, bias=True)
        #torch.nn.init.normal_(self.linear.weight, 0.0, 1e-3)

    def forward(self, x):
        # polynomial expansion
        phi = torch.stack([x[:, 0]**(k+1) for k in range(self.degree)], dim=1)
        # linear layer
        return self.linear(phi)

We train this model using the Mean Square Error loss, i.e.

> We assumme a Gaussian likelihood

and batch GD with adaptive learning rate and momentum (Adam)

With the `weight_decay` parameters of Adam we can add L2 regularization easily

> If we add L2 then we are obtaining MAP estimates with Gaussian likelihood and a Gaussian prior 

**Activity:**

1. Change the number of basis and describe the results
1. Increase the noise and repeat the previous step
1. Modify the `weight_decay` parameter in Adam and repeat the previous steps 

Concepts: Complexity, generalization, overfitting, regularization

In [ ]:
model = LinearRegressor(degree=10) # Change the degree
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=1e-2, 
                             weight_decay=0.0) # Change the weight decay

fig, ax = plt.subplots(1, 2, figsize=(7, 3), tight_layout=True)
f = model.forward(x_test).detach().numpy()
line1 = ax[0].plot(x_test.detach().numpy(), f, 'k-')
line2 = ax[1].plot([], [])
ax[0].scatter(x, y)

epoch_loss = np.zeros(shape=(10000,))
for k in tqdm_notebook(range(len(epoch_loss))):
    optimizer.zero_grad()
    f = model.forward(x_torch)
    loss = criterion(y_torch, f)
    loss.backward()
    #torch.nn.utils.clip_grad_value_(model.parameters(), 1e+1)
    optimizer.step()
    epoch_loss[k] = loss.item()
    #break    
    if k % 100 == 0:
        f = model.forward(x_test).detach().numpy()
        line1[0].set_ydata(f)
        line2[0].set_xdata(range(k))
        line2[0].set_ydata(epoch_loss[:k])
        for ax_ in ax:
            ax_.relim()
            ax_.autoscale_view()
        fig.canvas.draw()


TODO: Connect with the idea of basis functions

# Artificial Neural networks

[Artificial neural networks](https://docs.google.com/presentation/d/1IJ2n8X4w8pvzNLmpJB-ms6-GDHWthfsJTFuyUqHfXg8/edit?usp=sharing) (ANN) are non-linear parametric function approximators built by connecting **simple units**

These units are simplified models of biological neurons: 

> Linear regressor followed by a non-linear activation function

$$
y =  g \left( b + \sum_{d=1}^D w_{d} x_d  \right)
$$

with for example
$$
g(x) = \frac{1}{1 + e^{-x}}
$$

Feed-forward ANN are organized in layers. Each layer has a certain amount of neurons (user-defined)

> **Multilayer perceptron (MLP) architecture:** Every unit is connected to all units of its previous and next layers

For example a network with four inputs, two outputs and one **hidden layer** with eight neurons:



Mathematically, the output of the hidden layer is
$$
h_j =  g_1 \left( b_j + \sum_{d=1}^{N_x} w_{jd} x_d  \right), j=1, 2, \ldots, N_h
$$
and the output layer
$$
\begin{align}
f_i &=   g_2 \left(b_i + \sum_{j=1}^{N_h} w_{ij} h_j \right) \nonumber \\
&= g_2 \left(b_i + \sum_{j=1}^{N_h} w_{ij} g_1 \left( b_j + \sum_{d=1}^{N_x} w_{jd} x_d  \right) \right), i = 1, \ldots, N_o \nonumber
\end{align}
$$
where $g_1$, $g_2$ can be different

The parameters of the network are the weight and biases of the linear regressors

## MLP with Gaussian prior

The parameter vector $\theta$ includes the weights and biases of all the neurons

Let's consider a Gaussian prior for $\theta$ and study the space of possible models
- How does it compare to the single linear regressor? 
- What happens when you add more neurons? 
- What happens if you remove the nonlinearity? 
- What happens if you change the nonlinearity?
- What happens when you add more layers?

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)
x_nn = np.linspace(-5, 5, num=1000)[:, None].astype('float32') #100x1

class MLP(torch.nn.Module):
    def __init__(self, Nh=10, sw=5, sb=5):
        super(MLP, self).__init__()
        self.hidden1 = torch.nn.Linear(1, Nh)
        #self.hidden2 = torch.nn.Linear(Nh, Nh)
        #self.hidden3 = torch.nn.Linear(Nh, Nh)

        self.output = torch.nn.Linear(Nh, 1)
        #for layer in [self.hidden1, self.hidden2, self.hidden3, self.output]:
        for layer in [self.hidden1, self.output]:
            torch.nn.init.normal_(layer.weight, 0.0, sw)
            torch.nn.init.normal_(layer.bias, 0.0, sb)
        self.activation = torch.nn.Sigmoid()

    def forward(self, x):
        z = self.activation(self.hidden1(x))
        #z = self.activation(self.hidden2(z))
        #z = self.activation(self.hidden3(z))
        return self.output(z)
    
for i in range(10):
    model = MLP(Nh=1000)
    y_nn = model.forward(torch.from_numpy(x_nn)).detach().numpy()
    ax.plot(x_nn, y_nn, c='tab:blue', alpha=0.5)

## Probabilistic interpretation of ANN

Let's consider a simple MLP architecture for regression
- one hidden layer with $H$ neurons
- input dimensionality $D$ and output dimensionality $K$
- $g(\cdot)$ a nonlinear activation function (sigmoid, tanh, ReLU, etc)

$$
\begin{align}
f_i &=   b_i + \sum_{j=1}^H w_{ij} h_j  \nonumber \\
&=  b_i + \sum_{j=1}^H w_{ij} g \left( b_j + \sum_{d=1}^D w_{jd} x_d  \right) \nonumber
\end{align}
$$

The vector parameter $\vec \theta$ contains the weight and biases of both layers

We fit the parameters by minimizing the **Mean Square Error** cost function 

$$
\min_\theta \sum_n  \sum_i \left(y_{i}^{(n)} - f_i(x^{(n)}) \right)^2
$$

> This is equivalent to the **MLE solution with Gaussian likelihood** (known spherical covariance)

Typically an L2 regularizer is included to penalize complexity and improve generalization

$$
\min_\theta \sum_n  \sum_i  \left(y_{i}^{(n)} - f_i(x^{(n)}) \right)^2 + \lambda \sum_k \theta_k^2
$$

> This is equivalent to the **MAP solution with Gaussian likelihood and Gaussian prior** (zero-mean)

In both cases there is no closed-form solution and we optimize with iterative methods: **gradient descent**

## Long story short

> Conventional neural network training obtains MLE/MAP point estimates

For classification we arrive to the same conclusion except that 
- sigmoid or softmax activation is used in the output layer
- cross-entropy cost function is used instead of MSE: **Bernoulli/Categorical likelihood**

## Non-linear regressor using MLP

Let's go back to the polynomial regression problem

In [ ]:
# Synthetic data
fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)
ax.scatter(x, y);

This time we use a Multi-Layer Perceptron (MLP) neural network

We use a non-linear hyperbolic tangent activation function as hidden activation

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, n_hidden=10):
        super(MLP, self).__init__()
        self.hidden1 = torch.nn.Linear(1, n_hidden, bias=True)
        #self.hidden2 = torch.nn.Linear(n_hidden, n_hidden, bias=True)
        self.output = torch.nn.Linear(n_hidden, 1, bias=True)
        self.activation = torch.nn.Tanh()
        
    def forward(self, x):
        z = self.activation(self.hidden1(x))
        #return self.output(self.activation(self.hidden2(z)))
        return self.output(z)

How many hyperbolic tangent basis do we need to fit this data?

In [ ]:
model = MLP(n_hidden=10) # Change the number of hidden neurons

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=1e-2, 
                             weight_decay=0.0) # Change the weight decay

fig, ax = plt.subplots(1, 2, figsize=(7, 3), tight_layout=True, dpi=120)
f = model.forward(x_test).detach().numpy()
line1 = ax[0].plot(x_test.detach().numpy(), f, 'k-')
line2 = ax[1].plot([], [])
ax[0].scatter(x, y)

epoch_loss = np.zeros(shape=(5000,))
for k in tqdm_notebook(range(len(epoch_loss))):
    optimizer.zero_grad()
    f = model.forward(x_torch)
    loss = criterion(y_torch, f)
    loss.backward()
    optimizer.step()
    epoch_loss[k] = loss.item()
    #break    
    if k % 100 == 0:
        f = model.forward(x_test).detach().numpy()
        line1[0].set_ydata(f)
        line2[0].set_xdata(range(k))
        line2[0].set_ydata(epoch_loss[:k])
        for ax_ in ax:
            ax_.relim()
            ax_.autoscale_view()
        fig.canvas.draw()

# Deep Learning

> More complex and flexible models are obtained by increasing the number of hidden layers (depth) and the number of neurons (width) 



But 

> The more number of parameters, the more difficult to train: Overfitting, vanishing gradients, ...

With 
- Data availability on the rise 
- Faster hardware: GPUs (CUDA, tensor-cores)
- Regularization: penalty on parameters, data augmentation, dropout, ...
- Clever architectures and activations: Convolutional Neural Networks, Long-short term memories, ReLU, ...


> We are able to train **very deep neural network models** that have become the current state of the art in [pattern recognition problems](https://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#43494641522d3130)


**Extra:** Short tutorial on [Convolutional Neural Networks](https://github.com/magister-informatica-uach/INFO267/blob/master/unidad1/4_red_convolucional.ipynb)

#### Why are deep models needed?

The MLP with one hidden layer (shallow network) is a [universal approximator](http://cognitivemedium.com/magic_paper/assets/Hornik.pdf)

> (In theory) we could obtain a shallow network that is *as flexible* as a deep network


> (But) it may require an extremely large number of hidden-layer neurons (infinite)

In practice you need **flexible** but also **compact** models

## Next class

- Approximate Inference
- Our first Bayesian Neural network